In [1]:
import pandas as pd
import os
import numpy as np
import datetime as dt
from os import listdir
from os.path import isfile, join
from zebra import human_format

In [2]:
idx = pd.IndexSlice

In [3]:
def highlight_lost(x, color):
    return np.where(x == "Closed Lost", f"background: {color};", None )

***

# Data collection

## IT_Op_w_Products

In [5]:
op_w_prod = pd.read_csv(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Data export\IT_Op_w_Products.csv", encoding='latin-1', sep=";")

for col in op_w_prod.columns:
    if "Date" in col:
        op_w_prod[col] = op_w_prod[col].astype(str)
        op_w_prod[col] = pd.to_datetime(op_w_prod[col], format="%d/%m/%Y")
    else:
        pass
    
    
columns = [c for c in op_w_prod.columns if not "Currency" in c]
op_w_prod = op_w_prod.loc[:,columns]

num = []

for col in op_w_prod.columns:
    if "Amount" in col:
        num.append(col)
    else:
        pass
    
op_w_prod[num] = op_w_prod[num].apply(lambda x: x.str.replace(",",".", regex=False)).astype(float)

op_w_prod["weighted amount"] = op_w_prod["weighted amount"].apply(lambda x: x.replace(",",".")).astype(float)
op_w_prod["Total Value"] = op_w_prod["Total Value"].apply(lambda x: x.replace(",",".")).astype(float)

op_w_prod["Total Price"] = op_w_prod["Total Price"].astype(str).apply(lambda x: x.replace(",",".")).astype(float)

## IT_Opp_by_Created_Date

In [ ]:
op_by_date = pd.read_csv(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Salesforce\Data export\IT_Opp_by_Created_Date.csv", encoding='latin-1', sep=";")

op_by_date["Created Date"] = pd.to_datetime(op_by_date["Created Date"].astype(str), format="%d/%m/%Y")
op_by_date["Close Date"] = pd.to_datetime(op_by_date["Close Date"].astype(str), format="%d/%m/%Y")
op_by_date["Original Close Date"] = pd.to_datetime(op_by_date["Original Close Date"].astype(str), format="%d/%m/%Y")
op_by_date["Last Stage Change Date"] = pd.to_datetime(op_by_date["Last Stage Change Date"].astype(str), format="%d/%m/%Y")
op_by_date["Last Modified Date"] = pd.to_datetime(op_by_date["Last Modified Date"].astype(str), format="%d/%m/%Y")

columns = [c for c in op_by_date.columns if not "Currency" in c]
op_by_date = op_by_date[columns]

op_by_date.loc[:, ["Subscription Amount", "Non-Subscription Amount","Incremental Amount","weighted amount"]] = op_by_date.loc[:, ["Subscription Amount", "Non-Subscription Amount","Incremental Amount","weighted amount"]].apply(lambda x: x.str.replace(",",".", regex=False)).astype(float)

## Inflow

In [4]:
inflow_prodotti = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\inflow_prodotti\\"

onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\inflow_prodotti") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\inflow_prodotti", f))]
    
dfs = []
for file in onlyfiles:
    data = pd.read_excel(inflow_prodotti+file)
    dfs.append(data)
inflow = pd.concat(dfs, axis=0, ignore_index=True)
    
for col in inflow.select_dtypes(include=[object]).columns:
    inflow[col] = inflow[col].astype(str)

## Budget

In [58]:
budget_prodotti = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\budget_prodotti\\"

onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\budget_prodotti") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\budget_prodotti", f))]

dfs = []
for file in onlyfiles:
    data = pd.read_excel(budget_prodotti+file)
    dfs.append(data)
bdg = pd.concat(dfs, axis=0, ignore_index=True)
    
for col in bdg.select_dtypes(include=[object]).columns:
    bdg[col] = bdg[col].astype(str)

## Forecast

In [5]:
forecast = pd.read_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Source\forecast_sales_director\forecast_sales_director_2023.xlsx")

## Parametriche

In [6]:
parametriche = r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche\\"
onlyfiles = [f for f in listdir(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche") if isfile(join(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Parametriche", f))]

par = {}
for file in onlyfiles:
    file_name=file.split(".")[0]
    par[f"{file_name}"] = pd.read_excel(parametriche+file)


In [7]:
sales_director = par["Sales_director"][["Sales Director", "Rete", "Area territoriale"]]
linea_prodotti = par["Linea_prodotti"]

# Top five pending deals

In [77]:
now = dt.datetime.now()
current_month = now.month
current_year = now.year

In [79]:
opp_list = op_w_prod.loc[~op_w_prod["Stage"].str.contains("Closed", na=False),
                         ["Close Date", "Opportunity ID", "Opportunity Name","Stage","Ag MDM","Account Name", "Account Type","Owner Role","Product Name","Probability (%)","Subscription Amount","Non-Subscription Amount","Incremental Amount","weighted amount","Total Value","Total Price"]].copy()

opp_list.loc[:,opp_list.select_dtypes(include=['number']).columns] = opp_list.loc[:,opp_list.select_dtypes(include=['number']).columns].fillna(0)
opp_list.loc[:,opp_list.select_dtypes(include=['object']).columns] = opp_list.loc[:,opp_list.select_dtypes(include=['object']).columns].fillna("")

opp_list = opp_list.groupby([op_w_prod["Close Date"].astype('datetime64[M]'), "Opportunity ID", "Opportunity Name","Stage","Ag MDM","Account Name", "Account Type","Owner Role","Product Name"]).agg({"Probability (%)":"mean", "Subscription Amount":"max", "Non-Subscription Amount":"max","Incremental Amount":"max","weighted amount":"max","Total Value":"sum","Total Price":"sum"}).sort_values(by=["Incremental Amount"], ascending=False).reset_index()

opp_list["Products"] = opp_list.groupby(["Opportunity ID", "Account Name", "Account Type", "Stage", "Close Date", "Owner Role"])["Product Name"].transform(lambda x: " | ".join(x))
top_deals = opp_list.groupby(["Opportunity ID","Opportunity Name", "Account Name", "Account Type", "Products", "Stage", "Close Date", "Owner Role"], dropna=False).agg({"Incremental Amount":"max"}).reset_index().sort_values(by=["Incremental Amount"], ascending=False)

top_pend_deals = top_deals.loc[(top_deals["Close Date"].dt.date >= (dt.date(current_year,current_month,1))),:].copy()
top_pend_deals["Account Type"].replace({"Customer":"Existing","Prospect":"New"}, inplace=True)
top_pend_deals.columns = ["Opportunity ID", "Opportunity Name","Customer", "New / Existing client", "Product", "Stage", "Status (Closed/Pipeline/Lost/Postponed)", "Owner Role", "Size (€)"]
top_pend_deals.loc[:,["Business Unit", "Comments"]] = np.NaN 
top_pend_deals = top_pend_deals.loc[:,["Customer", "New / Existing client", "Business Unit", "Product", "Size (€)", "Comments", "Status (Closed/Pipeline/Lost/Postponed)", "Opportunity ID", "Opportunity Name", "Stage", "Owner Role"]]
top_pend_deals.index = np.arange(1, len(top_pend_deals)+1)

top_pend_deals_per_slide = top_pend_deals.loc[top_pend_deals["Status (Closed/Pipeline/Lost/Postponed)"].dt.date < dt.date(2023,12,31)].copy()
top_pend_deals_per_slide = top_pend_deals_per_slide[["Customer", "New / Existing client", "Business Unit", "Product", "Size (€)", "Comments", "Status (Closed/Pipeline/Lost/Postponed)"]]
top_pend_deals_per_slide["Status (Closed/Pipeline/Lost/Postponed)"] = top_pend_deals_per_slide["Status (Closed/Pipeline/Lost/Postponed)"].dt.strftime("Pipeline %b-%y")
top_pend_deals_per_slide.index = np.arange(1, len(top_pend_deals_per_slide)+1)
top_pend_deals_per_slide.index.set_names("#", inplace=True)

with pd.ExcelWriter(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Presentazioni\MDM\Top pending deals.xlsx") as writer:
    top_pend_deals_per_slide.to_excel(writer, sheet_name="top_pend_deals")
    top_pend_deals.to_excel(writer, sheet_name="descending_deals")
    opp_list.to_excel(writer, sheet_name="opp_list", index=False)

C:\Users\Raffaele.Sportiello\AppData\Local\Temp\1\ipykernel_20264\3655733436.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_pend_deals["Account Type"].replace({"Customer":"Existing","Prospect":"New"}, inplace=True)
C:\Users\Raffaele.Sportiello\AppData\Local\Temp\1\ipykernel_20264\3655733436.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_pend_deals.loc[:,["Business Unit", "Comments"]] = np.NaN


In [11]:
top_pend_deals.iloc[0:5, :].style.format(thousands=".", precision=0, subset=["Size (€)"]).format(formatter="{:%b, %Y}", subset=["Status (Closed/Pipeline/Lost/Postponed)"]).apply(highlight_lost, color="red")

,Customer,New / Existing client,Business Unit,Product,Size (€),Comments,Status (Closed/Pipeline/Lost/Postponed),Opportunity ID,Opportunity Name,Stage,Owner Role
1,CREDIT AGRICOLE GROUP SOLUTIONS SCPA,Existing,nan,,200.000,nan,"Mar, 2023",00607000003isni,AUTOMAZIONE IMPOSTE DIRETTE E INDIRETTE,Qualification,IT Large Account Sales Team
2,SALVATORE FERRAGAMO SPA,Existing,nan,PERSONALIZZAZIONI SOFTWARE,150.000,nan,"Feb, 2023",00607000001twWv,ferragamo,Qualification,IT Large Account Sales Team
3,UNICREDIT SPA,Existing,nan,,150.000,nan,"Feb, 2023",00607000003isn9,TCF,Qualification,IT Large Account Sales Team
4,TELECOM ITALIA SPA,Existing,nan,PROGETTO TAX PLUGIN 2.0 ATTIV.,150.000,nan,"Feb, 2023",00607000001tdtg,Opportunità Telecom,Qualification,IT Large Account Sales Team
5,Fincantieri SPA,New,nan,PROGETTO TAX PLUGIN 2.0 ATTIV. | ANAL.CONSUL. STUDIO GG INTERO AREA FISC.,120.000,nan,"Mar, 2023",00607000001t48K,Fincantieri SPA-,Qualification,IT Large Account Sales Team


In [42]:
top_deal_mese_consegnati = pd.DataFrame(data=["00607000001t47v", "00607000003iPml", "00607000003lfNJ", "00607000003kOAV", "00607000001tmek"], columns=["Opportunity ID"])


In [43]:
top_pend_deals.loc[top_pend_deals["Opportunity ID"].isin(top_deal_mese_consegnati["Opportunity ID"]),
                   ["Customer", "Size (€)", "Status (Closed/Pipeline/Lost/Postponed)", "Opportunity ID", "Opportunity Name", "Stage", "Owner Role"]]

,Customer,Size (€),Status (Closed/Pipeline/Lost/Postponed),Opportunity ID,Opportunity Name,Stage,Owner Role
17,INTESA SANPAOLO SPA,35000.0,2023-01-01,00607000003iPml,Intesa,Solution Presentation / Demo,IT Large Account Sales Team
21,ELIOS NEON SRL,27200.0,2023-01-01,00607000003kOAV,ARCA,Proposal,IT Enterprise Specialists Sales Team
23,Copan srl,20000.0,2023-01-01,00607000001tmek,Arca evolution - Copan srl,Solution Presentation / Demo,IT Enterprise Specialists Sales Team


<br>

***

# Last month sales intake performance – TAA Italy

In [7]:
columns = ["Opportunity ID","Opportunity Name", "Account Name",  "Owner Role", "Opportunity Owner", "Close Date","Stage", "Incremental Amount"]

now = dt.datetime.now()
prev_month = now.month-1
current_year = now.year

filt = (op_w_prod["Stage"].str.contains("Closed Won")) & (op_w_prod["Close Date"].dt.month==(prev_month)) & (op_w_prod["Close Date"].dt.year==(current_year))

closed_deals = op_w_prod.loc[filt,columns].drop_duplicates(subset=["Opportunity ID"]).sort_values(by="Incremental Amount", ascending=False)


filt = (op_w_prod["Stage"].str.contains("Closed Lost")) & (op_w_prod["Close Date"].dt.month==(prev_month)) & (op_w_prod["Close Date"].dt.year==(current_year))

lost_deals = op_w_prod.loc[filt,columns].drop_duplicates(subset=["Opportunity ID"]).sort_values(by="Incremental Amount", ascending=False)

In [44]:
with pd.ExcelWriter(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Presentazioni\MDM\Last month sales intake performance.xlsx") as writer:
    closed_deals.to_excel(writer, sheet_name="closed_deals", index=False)
    lost_deals.to_excel(writer, sheet_name="lost_deals", index=False)

### Success

In [11]:
now = dt.datetime.now()
prev_month = now.month-1
current_year = now.year

columns = ["Opportunity ID","Opportunity Name", "Product Name", "Total Price", "Account Name",  "Owner Role", "Close Date","Stage", "Incremental Amount"]

filt = (op_w_prod["Stage"].str.contains("Closed Won")) & (op_w_prod["Close Date"].dt.month==(prev_month)) & (op_w_prod["Close Date"].dt.year==(current_year))

closed_deals_con_prod = op_w_prod.loc[filt,:].groupby(["Opportunity ID","Opportunity Name","Account Name",  "Owner Role", "Close Date", "Product Name"]).agg({"Total Price":"sum","Incremental Amount":"max"}).sort_values(by=["Incremental Amount"], ascending=False)

closed_deals_con_prod.style.format("{:,.0f}€").background_gradient().to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Presentazioni\MDM\Success per MOR.xlsx")

# Pipeline – TAA Italy

## Pipeline

In [48]:
pipe = op_w_prod.drop_duplicates(subset=["Opportunity ID"])

### Total Pipeline

In [49]:
filt = (pipe["Close Date"].dt.month > (dt.datetime.now().month-1)) & (~pipe["Stage"].str.contains("Closed"))
tot_pipeline = pipe.loc[filt,:].groupby([pipe["Close Date"].astype('datetime64[M]')])[["Incremental Amount"]].sum().reset_index().pivot_table(columns="Close Date", values="Incremental Amount")

In [50]:
tot_pipeline

Close Date,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01
Incremental Amount,1744533.12,920046.33,261227.0,57863.0,42940.0,100500.0,32080.0,1800.0,0.0,8840.0


In [75]:
tot_pipeline.sum(axis=1)

Incremental Amount    3169829.45
dtype: float64

### Forecast 50<70 e Forecast >70

In [51]:
filt = (pipe["Close Date"].dt.month > (dt.datetime.now().month-1)) & (pipe["Ag MDM"].str.contains("70<90|50<70"))
pipe_50_90 = pipe.loc[filt,:].groupby([pipe["Close Date"].astype('datetime64[M]'), "Stage", "Ag MDM"])[[ "weighted amount"]].sum().unstack(0).fillna(0)

In [52]:
pipe_50_90

weighted amount                                   \
Close Date              2023-02-01 2023-03-01 2023-04-01 2023-05-01   
Stage       Ag MDM                                                    
Negotiation 70<90         66325.50    32881.8     4320.0        0.0   
Proposal    50<70        403042.94   104769.7    15397.9      201.6   

                                          
Close Date         2023-06-01 2023-09-01  
Stage       Ag MDM                        
Negotiation 70<90         0.0        0.0  
Proposal    50<70      7689.5    10780.0

### Actual

In [53]:
esc_fatture = par["Esclusione_fatture"]
esc_fatture["Numero Fattura"] = esc_fatture["Numero Fattura"].astype(str)

esc_ordini = par["Esclusione_ordini"]
esc_ordini["Numero Ordine"] = esc_ordini["Numero Ordine"].astype(str)

li_prodotti = par["Linea_prodotti"]

df_merge = inflow.merge(esc_fatture, on="Numero Fattura", how="left").merge(esc_ordini, on="Numero Ordine", how="left")

current_month = dt.date.today().replace(day=1)
filt = (df_merge["Data Ins. Ordine (monitoraggio)"].dt.date >= current_month) & (df_merge["Sales Director"].isin(["LARGE ACCOUNT","NORDEST IBISOFT","WKIC CENTRO SUD","WKIC NORD EST","WKIC NORD OVEST","ENTERPRISE"]))&(df_merge["Escludi fatture"].isna())&(df_merge["Escludi ordine"].isna())

In [54]:
actual = df_merge.loc[filt,:].groupby([df_merge["Data Ins. Ordine (monitoraggio)"].astype("datetime64[M]")]).agg({"Raccolto":"sum"}).transpose()

In [55]:
actual

Data Ins. Ordine (monitoraggio),2023-02-01,2023-03-01
Raccolto,151735.28,0.0


### Budget

In [56]:
sales_director["Sales Director"] = sales_director["Sales Director"].str.lower()

C:\Users\Raffaele.Sportiello\AppData\Local\Temp\1\ipykernel_20264\3000443029.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales_director["Sales Director"] = sales_director["Sales Director"].str.lower()


In [59]:
bdg["Sales Director"] = bdg["Sales Director"].str.lower()
budget = bdg.merge(sales_director, on="Sales Director", how="left").merge(linea_prodotti, on= "Linea Prodotti", how="left")
budget.rename(columns={"Data Ins. Ordine (monitoraggio)":"Data"}, inplace=True)

In [60]:
bdg_mid_month_forecast = budget.groupby(["Rete", "Linea", budget["Data"].astype('datetime64[M]')])["Budget"].sum().unstack(level=2).loc["Diretta"]
bdg_mid_month_forecast.style.format(human_format).background_gradient()

Data,2022-01-01 00:00:00,2022-02-01 00:00:00,2022-03-01 00:00:00,2022-04-01 00:00:00,2022-05-01 00:00:00,2022-06-01 00:00:00,2022-07-01 00:00:00,2022-08-01 00:00:00,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00
Linea,,,,,,,,,,,,
Servizi,231k,268k,346k,304k,308k,319k,317k,71k,315k,346k,346k,277k


In [61]:
tgt = budget.groupby(["Rete", budget["Data"].astype('datetime64[M]')])["Budget"].sum().unstack(level=1)
tgt.style.format(human_format).background_gradient()

Data,2022-01-01 00:00:00,2022-02-01 00:00:00,2022-03-01 00:00:00,2022-04-01 00:00:00,2022-05-01 00:00:00,2022-06-01 00:00:00,2022-07-01 00:00:00,2022-08-01 00:00:00,2022-09-01 00:00:00,2022-10-01 00:00:00,2022-11-01 00:00:00,2022-12-01 00:00:00
Rete,,,,,,,,,,,,
Diretta,231k,268k,346k,304k,308k,319k,317k,71k,315k,346k,346k,277k
Indiretta agenzie,110k,109k,110k,114k,134k,110k,144k,30.7k,163k,162k,166k,151k


### Forecast

In [62]:
sales = par["Sales_director"]
sales = sales.loc[:,["Sales Director", "Area territoriale", "Rete"]]
sales["Sales Director"] = sales["Sales Director"].apply(lambda x: x.lower())

forecast["Sales director"] = forecast["Sales director"].str.lower()
fc = forecast.merge(sales, left_on= "Sales director", right_on="Sales Director", how="left").merge(linea_prodotti, on= "Linea Prodotti", how="left")
fc = fc.loc[fc["Rete"] == "Diretta", ["Giorno","Linea", "Forecast"]].copy()
fc_month_forecast = fc.groupby(["Giorno", "Linea"]).agg({"Forecast":"sum"}).unstack(0)
fc_month_forecast.style.format(human_format).background_gradient()

In [25]:
fc_month_forecast.columns.get_level_values(1).to_period("M")


PeriodIndex(['2022-01', '2022-02', '2022-03', '2022-04', '2022-05', '2022-06',
             '2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12'],
            dtype='period[M]', name='Giorno')

### Total Pipeline FY

In [63]:
filt = (pipe["Close Date"].dt.month > (dt.datetime.now().month-1)) & (~pipe["Stage"].str.contains("Closed"))
tot_pipeline_fy = pipe.loc[filt,"Incremental Amount"].sum()
tot_pipeline_fy

3169829.45

## Export to excel

In [64]:
pipe_50_90_drop = pipe_50_90.droplevel(level=0, axis=1)
pipe_50_90_drop = pipe_50_90_drop.droplevel(level=0, axis=0)

In [65]:
pipeline_taa_italy = pd.concat([tot_pipeline, pipe_50_90_drop,  bdg_mid_month_forecast])

In [66]:
pipeline_taa_italy

,2022-01-01,2022-02-01,2022-03-01,2022-04-01,2022-05-01,2022-06-01,2022-07-01,2022-08-01,2022-09-01,2022-10-01,...,2023-02-01,2023-03-01,2023-04-01,2023-05-01,2023-06-01,2023-07-01,2023-09-01,2023-10-01,2023-11-01,2023-12-01
Incremental Amount,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1744533.12,920046.33,261227.0,57863.0,42940.0,100500.0,32080.0,1800.0,0.0,8840.0
70<90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,66325.50,32881.80,4320.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN
50<70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,403042.94,104769.70,15397.9,201.6,7689.5,NaN,10780.0,NaN,NaN,NaN
Servizi,230783.353272,267555.591522,346365.0,303907.447081,307614.451856,318599.407082,316502.384779,70996.512972,315427.139243,346365.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
pipeline_taa_italy.to_excel(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Presentazioni\MDM\Pipeline.xlsx")

# Sales KPIs

In [71]:
filt_rows = (inflow["Data Ins. Ordine (monitoraggio)"].dt.year == current_year)&(inflow["Raccolto"].notna())
columns = ["Cliente Merce", "Sales Director", "Canale di VENDITA", "Linea Prodotti", "Numero Ordine", "Numero Fattura", "Data Ins. Ordine (monitoraggio)", "Raccolto"]

df_2022 = inflow.loc[filt_rows, columns].reset_index(drop=True)
df_2022["Sales Director"] = df_2022["Sales Director"].str.lower()

esc_fatture = par["Esclusione_fatture"]
esc_fatture["Numero Fattura"] = esc_fatture["Numero Fattura"].astype(str)

esc_ordini = par["Esclusione_ordini"]
esc_ordini["Numero Ordine"] = esc_ordini["Numero Ordine"].astype(str)

sales_director = par["Sales_director"]
sales_director["Sales Director"] = sales_director.loc[:,"Sales Director"].str.lower()
sales_director = sales_director[["Sales Director", "Rete", "Rete2", "Area territoriale"]]

li_prodotti = par["Linea_prodotti"]

inflow_sKPI = df_2022.merge(esc_fatture, on="Numero Fattura", how="left").merge(esc_ordini, on="Numero Ordine", how="left").merge(li_prodotti, on="Linea Prodotti", how="left").merge(sales_director, on="Sales Director", how="left")

esclusioni = (inflow_sKPI["Canale di VENDITA"] != "E-Commerce")&(inflow_sKPI["Escludi fatture"].isna())&(inflow_sKPI["Escludi ordine"].isna())

inflow_sKPI = inflow_sKPI.loc[esclusioni, ['Data Ins. Ordine (monitoraggio)', "Cliente Merce", "Numero Ordine", "Rete", "Rete2","Area territoriale", 'Sales Director', 'Linea', 'Raccolto']].reset_index(drop=True)

inflow_sKPI.rename(columns={"Data Ins. Ordine (monitoraggio)":'Giorno'}, inplace=True)

#### Ordine medio per cliente  
Consideriamo l'ordine medio per cliente, anche se nella slide c'è scritto "Average deal size"

In [72]:
inflow_sKPI["N. Clienti"] = inflow_sKPI.groupby(["Rete2", inflow_sKPI["Giorno"].astype('datetime64[M]')], dropna=False)["Cliente Merce"].transform("nunique")
inflow_gr = inflow_sKPI.groupby(["Rete2", inflow_sKPI["Giorno"].astype('datetime64[M]')], dropna=False).agg({"Raccolto":"sum", "N. Clienti":"max"})

inflow_gr["Avg deal size"] = inflow_gr["Raccolto"] / inflow_gr["N. Clienti"]

inflow_gr["Avg deal size"].unstack(level=1).style.format("{:,.0f}").background_gradient()

Giorno,2023-01-01 00:00:00,2023-02-01 00:00:00
Rete2,,
Diretta + Indiretta agenzie,959,940
Indiretta concessionari,"1,156",576


# Sales Quota by Sales Rep and Sales Team (direct channel only)

In [33]:
periodo = (inflow["Data Ins. Ordine (monitoraggio)"].dt.year == 2022) & (inflow["Data Ins. Ordine (monitoraggio)"].dt.quarter.isin([1,2]))
filt_rows = (periodo)

columns = ["Cliente Merce", "Sales Director", "Agente dell'ORDINE", "Canale di VENDITA", "Linea Prodotti", "Numero Ordine", "Numero Fattura", "Data Ins. Ordine (monitoraggio)", "Raccolto", "Budget  assegnato"]

df2 = inflow.loc[filt_rows, columns].reset_index(drop=True)
df2["Sales Director"] = df2["Sales Director"].str.lower()

esc_fatture = par["Esclusione_fatture"]
esc_fatture["Numero Fattura"] = esc_fatture["Numero Fattura"].astype(str)

esc_ordini = par["Esclusione_ordini"]
esc_ordini["Numero Ordine"] = esc_ordini["Numero Ordine"].astype(str)

sales_director = par["Sales_director"]
sales_director["Sales Director"] = sales_director.loc[:,"Sales Director"].str.lower()
sales_director = sales_director[["Sales Director", "Rete"]]

li_prodotti = par["Linea_prodotti"]

agente_ordine = par["Agente_ordine"]
agente_ordine.rename(columns = {"Agente dell'ORDINE":"Sales rep"}, inplace=True) 

sales_quota = df2.merge(esc_fatture, on="Numero Fattura", how="left").merge(esc_ordini, on="Numero Ordine", how="left").merge(li_prodotti, on="Linea Prodotti", how="left").merge(sales_director, on="Sales Director", how="left").merge(agente_ordine, left_on="Agente dell'ORDINE", right_on="Codice agente", how="left")

esclusioni = (sales_quota["Canale di VENDITA"] != "E-Commerce") & (sales_quota["Escludi fatture"].isna()) & (sales_quota["Escludi ordine"].isna()) & (sales_quota["Rete"] == "Diretta") & (sales_quota["Sales Director"] != "large account")

sales_quota = sales_quota.loc[esclusioni, ['Data Ins. Ordine (monitoraggio)', "Codice agente", "Sales rep", 'Raccolto', "Budget  assegnato"]].reset_index(drop=True)

sales_quota.rename(columns={"Data Ins. Ordine (monitoraggio)":'Giorno'}, inplace=True)

In [34]:
sales_quota.groupby(["Sales rep", sales_quota["Giorno"].astype("datetime64[M]")]).agg({"Raccolto":"sum", "Budget  assegnato":"sum"}).loc[idx[["BONDI GIAMMARCO", "DI IENNO MONICA"],:],:].style.format("{:,.0f}").background_gradient()

In [35]:
sales_quota_gr = sales_quota.groupby(["Sales rep"]).agg({"Raccolto":"sum", "Budget  assegnato":"sum"}).reset_index().sort_values(by=["Budget  assegnato"], ascending=False)

# Sales Activity

In [36]:
sales_activity = op_w_prod.loc[(op_w_prod["Created Date"].dt.date >=dt.date(2022,1,1)) & (op_w_prod["Created Date"].dt.quarter.isin([1,2,3,4])),:].copy()
sales_activity["Created Week"] = op_w_prod["Created Date"].dt.isocalendar().week
sales_activity["Created Quarter"] = op_w_prod["Created Date"].dt.quarter

In [37]:
sales_activity_by_week = sales_activity.groupby(["Created Week", "Created Quarter"]).agg({"Opportunity ID":"nunique"}).reset_index()

sales_activity_by_day = sales_activity.groupby(["Created Date","Created Week", "Created Quarter"]).agg({"Opportunity ID":"nunique"}).reset_index()

In [38]:
with pd.ExcelWriter(r"C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Presentazioni\MDM\Sales Quota & Sales Activity.xlsx", engine="openpyxl") as writer:
    sales_quota_gr.to_excel(writer, sheet_name="sales_quota", index=False)
    sales_activity_by_week.to_excel(writer, sheet_name="op_by_week", index=True)
    #sales_activity_by_day.to_excel(writer, sheet_name="op_by_day", index=True)

# Clienti SMART Skills

In [12]:
from directories import *

In [18]:
par = {}
for file in parametriche_files:
    file_name=file.split(".")[0]
    par[f"{file_name}"] = pd.read_excel(parametriche+"\\"+file)

In [24]:
dfs = []
for file in dashboard_inflow_prodotti_files:
    data = pd.read_excel(dashboard_inflow_prodotti+"\\"+file)
    dfs.append(data)
inflow = pd.concat(dfs, axis=0, ignore_index=True)

for col in inflow.select_dtypes(include=[object]).columns:
    inflow[col] = inflow[col].astype(str)

In [26]:
columns = ["Cliente Merce", "Sales Director", "Canale di VENDITA", "Linea Prodotti", "Codice PRODOTTO", "Numero Ordine", "Numero Fattura", "Data Ins. Ordine (monitoraggio)", "Raccolto"]

df_smart = inflow.loc[:, columns].reset_index(drop=True)

df_smart["Sales Director"] = df_smart["Sales Director"].str.lower()
df_smart["Codice PRODOTTO"] = df_smart["Codice PRODOTTO"].astype(str)

esc_fatture = par["Esclusione_fatture"]
esc_fatture["Numero Fattura"] = esc_fatture["Numero Fattura"].astype(str)

esc_ordini = par["Esclusione_ordini"]
esc_ordini["Numero Ordine"] = esc_ordini["Numero Ordine"].astype(str)

sales_director = par["Sales_director"]
sales_director["Sales Director"] = sales_director.loc[:,"Sales Director"].str.lower()
sales_director = sales_director[["Sales Director", "Rete", "Rete3", "Area territoriale"]]

prodotti = par["Classificazione_codice_prodotto"]
prodotti["Codice PRODOTTO"] = prodotti["Codice PRODOTTO"].astype(str)

li_prodotti = par["Linea_prodotti"]

clienti_smart = df_smart.merge(esc_fatture, on="Numero Fattura", how="left").merge(esc_ordini, on="Numero Ordine", how="left").merge(li_prodotti, on="Linea Prodotti", how="left").merge(sales_director, on="Sales Director", how="left").merge(prodotti, on="Codice PRODOTTO", how="left")

esclusioni = (clienti_smart["Escludi fatture"].isna())&(clienti_smart["Escludi ordine"].isna())&(clienti_smart["Sub solution"] == "Smart - Skills")

clienti_smart = clienti_smart.loc[esclusioni, ['Data Ins. Ordine (monitoraggio)', "Cliente Merce", "Numero Ordine", "Rete3", "Canale di VENDITA",'Linea', 'Raccolto']].reset_index(drop=True)

clienti_smart.rename(columns={"Data Ins. Ordine (monitoraggio)":'Giorno'}, inplace=True)

## Numero clienti

In [47]:
arr = clienti_smart.loc[clienti_smart["Giorno"].dt.year == current_year, "Giorno"].astype('datetime64[M]').sort_values().unique()
d_rete = {}
d_canale = {}

for i,elem in enumerate(arr):
    temp = clienti_smart.loc[(clienti_smart["Giorno"].astype('datetime64[M]').dt.date >= dt.date(current_year,1,1)) &
                             (clienti_smart["Giorno"].astype('datetime64[M]') <= elem) &
                             (clienti_smart["Canale di VENDITA"] != "E-Commerce"), :].groupby(["Rete3"]).agg({"Cliente Merce":"nunique"}).reset_index()
    d_rete[f"d_{i}"] = temp["Cliente Merce"].to_numpy()

    temp = clienti_smart.loc[(clienti_smart["Giorno"].astype('datetime64[M]').dt.date >= dt.date(current_year,1,1)) &
                             (clienti_smart["Giorno"].astype('datetime64[M]') <= elem) &
                             (clienti_smart["Canale di VENDITA"] == "E-Commerce"), :].groupby(["Canale di VENDITA"]).agg({"Cliente Merce":"nunique"}).reset_index()
    d_canale[f"d_{i}"] = temp["Cliente Merce"].to_numpy()


In [49]:
df_rete = pd.DataFrame.from_dict(data= d_rete.items())
df_rete = pd.DataFrame(df_rete[1].tolist())
df_rete = df_rete.pivot_table(columns=df_rete.index)

df_canale = pd.DataFrame.from_dict(data= d_canale.items())
df_canale = pd.DataFrame(df_canale[1].tolist())
df_canale = df_canale.pivot_table(columns=df_canale.index)

clienti_unici = pd.concat([df_rete, df_canale], axis=0)
clienti_unici.columns = arr
clienti_unici.index = ["Diretta", "Indiretta", "E-commerce"]
clienti_unici = clienti_unici.transpose()
clienti_unici["Traditional channels"] = clienti_unici["Diretta"] + clienti_unici["Indiretta"]
clienti_unici = clienti_unici.loc[dt.date(current_year,1,1):, ["Traditional channels", "E-commerce"]]

In [51]:
inflow_progr_Anna = clienti_smart.loc[(clienti_smart["Canale di VENDITA"] != "E-Commerce") & 
                                      (clienti_smart["Giorno"].dt.date >= dt.date(current_year,1,1)), 
                                      :].groupby([clienti_smart["Giorno"].astype("datetime64[M]")])["Raccolto"].sum().reset_index()
inflow_progr_Anna["Anna"] = inflow_progr_Anna["Raccolto"].cumsum()

inflow_progr_Ben = clienti_smart.loc[(clienti_smart["Canale di VENDITA"] == "E-Commerce") &
                                    (clienti_smart["Giorno"].dt.date >= dt.date(current_year,1,1)), 
                                    :].groupby([clienti_smart["Giorno"].astype("datetime64[M]")])["Raccolto"].sum().reset_index()
inflow_progr_Ben["Ben"] = inflow_progr_Ben["Raccolto"].cumsum()

inflow_progr = inflow_progr_Anna.merge(inflow_progr_Ben, how="outer", on="Giorno")
inflow_progr = inflow_progr.loc[:,["Giorno","Anna","Ben"]]

In [ ]:
with pd.ExcelWriter(onedrive_documents + r"\Presentazioni\MDM\Smart SKILL\clienti_e_inflow_per_slide.xlsx", engine="openpyxl") as writer:
    inflow_progr.to_excel(writer, sheet_name="inflow_progr", index=False)
    clienti_unici.to_excel(writer, sheet_name="clienti_unici", index=True)